In [10]:
##### This Jupyter notebook generates MHLDA CVs. 
##### Please ensure you have properly set up the conda environment with all libraries.
##### Please note that this code could be simplified using the scikit-learn library. 
##### Additionally, the code by Mendels et al. is available for download from their GitHub. However, the code presented here was written from scratch.

##### Author: MO (latest update: May 28, 2024)

In [11]:
##### User Inputs #####
nDataPoints = 754 # Number of data points in each class (*note: each class should have the same number of data points)
num_class = 3 # Number of classes
num_descriptor = 7 # Number of descriptors or features
num_eigenvector = 2 # Number of eigenvectors or CVs (reduced dimensionality)
descriptor_list = ['res159.439', 'res245.369', 'res64.137', 'res199.471', 'res78.450', 'res242.340', 'res77.293'] # List of feature names

In [12]:
### STEP 0. Import libraries
import pandas as pd
import numpy as np

In [13]:
### STEP 1. Load input data
df = pd.read_csv('mpso.csv')

In [14]:
### STEP 2. Zero-mean the data
np.set_printoptions(precision=8)
for elem in descriptor_list:
    print('Mean for ', elem, ': ', df[elem].mean())
    df[elem] = df[elem] - df[elem].mean()

Mean for  res159.439 :  21.74415728372237
Mean for  res245.369 :  38.09927344891246
Mean for  res64.137 :  16.775558394920427
Mean for  res199.471 :  30.131755684036246
Mean for  res78.450 :  13.889633035910256
Mean for  res242.340 :  26.819251453872678
Mean for  res77.293 :  40.03164760639258


In [15]:
### STEP 3. Separate data and generate labels
X = df.iloc[:,:num_descriptor].values
X = X.astype(np.float64)
y = np.concatenate([np.zeros(nDataPoints)+1,np.ones(nDataPoints)+1,np.ones(nDataPoints)+2])
print(X)
print(y)

[[-0.76525385  2.08614583  1.03468783 ...  0.23212689  1.08992405
   0.75515021]
 [-1.47115657  2.44363145  0.69819475 ...  2.78658885  1.9426788
   2.18380523]
 [-2.18411414  2.18471138 -0.08038096 ...  1.60488209  1.93621906
   1.18560777]
 ...
 [ 2.30423513 -2.98455501  2.50959053 ... -3.00694806 -2.71507559
  -1.41571489]
 [ 3.33669351 -3.31498275  2.7419514  ... -3.27287224 -3.22628615
  -1.51252031]
 [ 1.98100839 -3.30819255  2.2219749  ... -2.74602217 -3.0444609
  -1.75634772]]
[1. 1. 1. ... 3. 3. 3.]


In [16]:
### STEP 4. Compute the d-dimensional mean vectors
### Here, we calculate #num_class column vectors, each of which contains #num_descriptor elements (means)
np.set_printoptions(precision=4)
mean_vectors = []
for cl in range(1,num_class+1):
    mean_vectors.append(np.mean(X[y==cl], axis=0))                
    print(f'Mean Vector class {cl}: {mean_vectors[cl-1]}')

Mean Vector class 1: [-0.522   1.7285  0.7063  2.3102  3.008   1.4236  0.4796]
Mean Vector class 2: [-1.0786  1.446  -3.2836 -0.1895  0.5859  0.4779  2.1154]
Mean Vector class 3: [ 1.6007 -3.1745  2.5773 -2.1207 -3.5939 -1.9015 -2.595 ]


In [17]:
### STEP 5. Compute the scatter matrices
### 5-1. Within-class scatter matrix SW
S_W = np.zeros((num_descriptor,num_descriptor))
S_W_int = np.zeros((num_descriptor,num_descriptor))
for cl,mv in zip(range(1,num_class+1), mean_vectors):
    class_sc_mat = np.zeros((num_descriptor,num_descriptor))
    for row in X[y==cl]:
        row, mv = row.reshape(num_descriptor,1), mv.reshape(num_descriptor,1)   # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    S_W_int += np.linalg.inv(class_sc_mat)                                      # sum class scatter matrices
S_W = np.linalg.inv(S_W_int)

print('within-class Scatter Matrix:\n', S_W)

within-class Scatter Matrix:
 [[110.2203 -17.0821   1.0315   4.7739  22.694  -18.65   -28.9193]
 [-17.0821  98.2108  -5.0467  60.9005   8.6566  45.9013  17.0057]
 [  1.0315  -5.0467  66.8939  -2.7538  -1.518    0.9575  -1.2825]
 [  4.7739  60.9005  -2.7538 188.8873  22.2253  30.0033  27.5583]
 [ 22.694    8.6566  -1.518   22.2253  88.9301   2.5322   8.1295]
 [-18.65    45.9013   0.9575  30.0033   2.5322  90.7032  11.1465]
 [-28.9193  17.0057  -1.2825  27.5583   8.1295  11.1465 161.0285]]


In [18]:
### 5-2. Between-class scatter matrix SB
overall_mean = np.mean(X, axis=0)                               
S_B = np.zeros((num_descriptor,num_descriptor))
for i,mean_vec in enumerate(mean_vectors):                      
    n = X[y==i+1,:].shape[0]                                    
    mean_vec = mean_vec.reshape(num_descriptor,1)               
    overall_mean = overall_mean.reshape(num_descriptor,1)       
    S_B += n*(mean_vec-overall_mean).dot((mean_vec-overall_mean).T)
    
print('between-class Scatter Matrix:\n', S_B)

between-class Scatter Matrix:
 [[  3014.6095  -5687.7397   5503.1252  -3314.7076  -5997.9838  -3244.0209
   -5041.2023]
 [ -5687.7397  11427.8272  -8828.4601   7880.4565  13161.463    6927.9563
    9142.848 ]
 [  5503.1252  -8828.4601  13514.3623  -2421.6046  -6832.5517  -4120.3529
  -10024.9322]
 [ -3314.7076   7880.4565  -2421.6046   7442.3423  10902.6679   5452.0663
    4682.6165]
 [ -5997.9838  13161.463   -6832.5517  10902.6679  16819.8248   8592.6645
    9054.2095]
 [ -3244.0209   6927.9563  -4120.3529   5452.0663   8592.6645   4426.6344
    4997.7325]
 [ -5041.2023   9142.848  -10024.9322   4682.6165   9054.2095   4997.7325
    8625.1312]]


In [19]:
### STEP 6. Solve the generalized eigenvalue problem for the matrix SW^-1.SB
eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
for i in range(len(eig_vals)):
    eigvec_sc = eig_vecs[:,i].reshape(num_descriptor,1)         # [:,i] = all rows and column i
    print(f'\nEigenvector {i+1}: \n{eigvec_sc.real}')
    print(f'Eigenvalue {i+1}: {eig_vals[i].real:.2e}')

for i in range(len(eig_vals)):
    eigv = eig_vecs[:,i].reshape(num_descriptor,1)
    np.testing.assert_array_almost_equal(np.linalg.inv(S_W).dot(S_B).dot(eigv), eig_vals[i] * eigv, decimal=6, err_msg='', verbose=True)
print('Good')


Eigenvector 1: 
[[ 0.2437]
 [-0.3491]
 [ 0.6375]
 [ 0.0645]
 [-0.6148]
 [-0.08  ]
 [-0.1543]]
Eigenvalue 1: 4.16e+02

Eigenvector 2: 
[[-0.9585]
 [-0.1012]
 [ 0.2058]
 [ 0.0246]
 [-0.1589]
 [-0.0182]
 [-0.0497]]
Eigenvalue 2: 1.48e-14

Eigenvector 3: 
[[-0.1369]
 [-0.0383]
 [ 0.7533]
 [ 0.1983]
 [ 0.5633]
 [ 0.1543]
 [-0.1785]]
Eigenvalue 3: 1.33e+02

Eigenvector 4: 
[[ 0.9285]
 [ 0.2152]
 [-0.0641]
 [-0.1099]
 [ 0.007 ]
 [ 0.2188]
 [ 0.1656]]
Eigenvalue 4: 1.96e-14

Eigenvector 5: 
[[ 0.0553]
 [-0.3093]
 [-0.2437]
 [ 0.1811]
 [-0.1897]
 [ 0.7006]
 [-0.2281]]
Eigenvalue 5: -1.10e-14

Eigenvector 6: 
[[ 0.0553]
 [-0.3093]
 [-0.2437]
 [ 0.1811]
 [-0.1897]
 [ 0.7006]
 [-0.2281]]
Eigenvalue 6: -1.10e-14

Eigenvector 7: 
[[-0.8927]
 [ 0.1059]
 [ 0.1792]
 [-0.2328]
 [ 0.0594]
 [-0.2121]
 [-0.2388]]
Eigenvalue 7: -3.76e-15
Good


In [20]:
### STEP 7. Select linear discriminants for the new feature subspace
### 7-1. Sort the eigenvectors by decreasing eigenvalues
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]    # make a list of (eigenvalue, eigenvector) tuples
eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)                     # sort the (eigenvalue, eigenvector) tuples from high to low

print('Eigenvalues in decreasing order:\n')     # visually confirm that the list is correctly sorted by decreasing eigenvalues
for i in eig_pairs:
    print(i[0])

print('Variance explained:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print(f'eigenvalue {i+1}: {(j[0]/eigv_sum).real:.2%}')

W = np.concatenate([eig_pairs[i][1].reshape(num_descriptor,1) for i in range(num_eigenvector)], axis=1)
print('Matrix W:\n', W.real)

### STEP 8. Transform the samples onto the new subspace
X_ldaz = X.dot(W.real) 
y = np.concatenate([np.zeros(nDataPoints),np.ones(nDataPoints),np.ones(nDataPoints)+1])

np.savetxt("MHLDA.csv", X_ldaz, delimiter=",", fmt="%.5f", header="LD1, LD2", comments="")
df2 = pd.read_csv('MHLDA.csv')
df2['class'] = y
df2.to_csv('MHLDA.csv', encoding='utf-8', index=False)

Eigenvalues in decreasing order:

416.3941828430634
133.35437120929552
1.959256755023652e-14
1.7457549457945934e-14
1.7457549457945934e-14
1.4800029971683448e-14
3.7605881195825926e-15
Variance explained:

eigenvalue 1: 75.74%
eigenvalue 2: 24.26%
eigenvalue 3: 0.00%
eigenvalue 4: 0.00%
eigenvalue 5: 0.00%
eigenvalue 6: 0.00%
eigenvalue 7: 0.00%
Matrix W:
 [[ 0.2437 -0.1369]
 [-0.3491 -0.0383]
 [ 0.6375  0.7533]
 [ 0.0645  0.1983]
 [-0.6148  0.5633]
 [-0.08    0.1543]
 [-0.1543 -0.1785]]
